# TAGgle
A wordle-like based on imdb film keywords.

The code here is scrapes data from IMDB and writes it to a json for use in the external app. 

The only variable needed to be altered here is the size, which references how many pages to scrape. Each page is 50 films. This enables the external app to choose from several difficulty levels.


In [1]:
#import dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import re
import fnmatch
import random
import string
import json

In [2]:

# how many pages to return, at 50 films per page
# this effectively sets a difficulty level; 1: easy, 5: medium, 20: severe
size = 60


In [3]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 105.0.5195
Get LATEST chromedriver version for 105.0.5195 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\David\.wdm\drivers\chromedriver\win32\105.0.5195.52]


In [4]:
# method to grab all 50 films from an IMDB URL
def populate_films(url):
    browser.visit(url)
    soup = bs(browser.html, 'html.parser')
    # the image next to the film contains most of the data we need: film title in the alt-text, and film id in 'data-tconst'.
    href_list = soup.find_all('img', class_='loadlate')
    for item in href_list:
        film_names.append(item['alt'])
        film_ids.append(item['data-tconst'])
#     now grab the year of the film in case of name conflict
    year_list = soup.find_all('span', class_='lister-item-year text-muted unbold')
    for item in year_list:
        years.append(item.contents[0])

In [5]:
film_names = []
film_ids = []
years = []

print(f"grabbing the names and id of the first {size*50} films, ranked by most rated")    
for i in range (0,size):
    url = f"https://www.imdb.com/search/keyword/?ref_=kw_nxt&mode=detail&page={i+1}&sort=num_votes,desc&title_type=movie"
    populate_films(url)
    print(f"getting films from page {i+1} of {size}")
    

grabbing the names and id of the first 3000 films, ranked by most rated
getting films from page 1 of 60
getting films from page 2 of 60
getting films from page 3 of 60
getting films from page 4 of 60
getting films from page 5 of 60
getting films from page 6 of 60
getting films from page 7 of 60
getting films from page 8 of 60
getting films from page 9 of 60
getting films from page 10 of 60
getting films from page 11 of 60
getting films from page 12 of 60
getting films from page 13 of 60
getting films from page 14 of 60
getting films from page 15 of 60
getting films from page 16 of 60
getting films from page 17 of 60
getting films from page 18 of 60
getting films from page 19 of 60
getting films from page 20 of 60
getting films from page 21 of 60
getting films from page 22 of 60
getting films from page 23 of 60
getting films from page 24 of 60
getting films from page 25 of 60
getting films from page 26 of 60
getting films from page 27 of 60
getting films from page 28 of 60
getting films

In [6]:
# print a list of all the films 
for i in range (0, len(film_names)):
    print(f"{i+1}: {film_ids[i]}, {film_names[i]}, {years[i]}")

1: tt0111161, The Shawshank Redemption, (1994)
2: tt0468569, The Dark Knight, (2008)
3: tt1375666, Inception, (2010)
4: tt0137523, Fight Club, (1999)
5: tt0109830, Forrest Gump, (1994)
6: tt0110912, Pulp Fiction, (1994)
7: tt0133093, The Matrix, (1999)
8: tt0120737, The Lord of the Rings: The Fellowship of the Ring, (2001)
9: tt0068646, The Godfather, (1972)
10: tt0167260, The Lord of the Rings: The Return of the King, (2003)
11: tt0816692, Interstellar, (2014)
12: tt1345836, The Dark Knight Rises, (2012)
13: tt0167261, The Lord of the Rings: The Two Towers, (2002)
14: tt0114369, Seven, (1995)
15: tt1853728, Django Unchained, (2012)
16: tt0172495, Gladiator, (2000)
17: tt0372784, Batman Begins, (2005)
18: tt0361748, Inglourious Basterds, (2009)
19: tt0102926, The Silence of the Lambs, (1991)
20: tt0993846, The Wolf of Wall Street, (2013)
21: tt0848228, Avengers Assemble, (2012)
22: tt0120815, Saving Private Ryan, (1998)
23: tt0076759, Star Wars, (1977)
24: tt0108052, Schindler's List, 

2128: tt0480242, Dan in Real Life, (I) (2007)
2129: tt0412019, Broken Flowers, (2005)
2130: tt3622592, Paper Towns, (2015)
2131: tt1027718, Wall Street: Money Never Sleeps, (2010)
2132: tt11245972, Scream, (I) (2022)
2133: tt0209958, The Cell, (2000)
2134: tt0111495, Three Colours: Red, (1994)
2135: tt0114924, While You Were Sleeping, (1995)
2136: tt4572514, Patriots Day, (2016)
2137: tt4686844, The Death of Stalin, (2017)
2138: tt0080120, The Warriors, (1979)
2139: tt2224026, Home, (II) (2015)
2140: tt4465564, Fifty Shades Darker, (2017)
2141: tt0213847, Malena, (2000)
2142: tt3381008, Grimsby, (2016)
2143: tt0349205, Cheaper by the Dozen, (2003)
2144: tt1623288, ParaNorman, (2012)
2145: tt1614989, Headhunters, (2011)
2146: tt0066206, Patton, (1970)
2147: tt0477302, Extremely Loud & Incredibly Close, (2011)
2148: tt0386140, The Legend of Zorro, (2005)
2149: tt0094862, Child's Play, (1988)
2150: tt0097216, Do the Right Thing, (1989)
2151: tt0441909, Volver, (I) (2006)
2152: tt0108394, 

In [7]:
# this method scrapes the keywords from the indivual page for the given film id.
def get_keywords(film_id):
    keyURL = f"https://www.imdb.com/title/{film_id}/keywords"
    browser.visit(keyURL)
    soup = bs(browser.html, 'html.parser')
    # in the keywords table, each keyword can be found in a 'sodatext' div.
    key_list = soup.find_all('div', class_='sodatext')
    
    keys = []
    for div in key_list:
        # we just want the text from what is in the page as an anchor.
        keys.append(div.find('a').contents[0])
    return keys
    

In [8]:
# code to remove punctutation and capitals from any text.
# perhaps common words like 'the' should be removed to?

def SPAG_remover(word):
    return word.lower().translate(str.maketrans('', '', string.punctuation)).replace('·', '').replace('the ', '').replace(' and ', '').replace(' ', '')

In [9]:
# this method loops through the film names, and for each one, calls the 'get_keywords' method. 
# all this data is then wrapped up in a dictionary.
# the dictionary is then added to the main dictionary, with a cleaned-up film name.
film_dict = {}
for i in range(0, len(film_names)):
    print(f"film {i+1} of {len(film_names)}")
    new_film = {'id': film_ids[i], 'punc_name': film_names[i] + " " + years[i], 'tags': get_keywords(film_ids[i])}
    film_dict.update({(SPAG_remover(film_names[i])) + years[i]: new_film})

film 1 of 3000
film 2 of 3000
film 3 of 3000
film 4 of 3000
film 5 of 3000
film 6 of 3000
film 7 of 3000
film 8 of 3000
film 9 of 3000
film 10 of 3000
film 11 of 3000
film 12 of 3000
film 13 of 3000
film 14 of 3000
film 15 of 3000
film 16 of 3000
film 17 of 3000
film 18 of 3000
film 19 of 3000
film 20 of 3000
film 21 of 3000
film 22 of 3000
film 23 of 3000
film 24 of 3000
film 25 of 3000
film 26 of 3000
film 27 of 3000
film 28 of 3000
film 29 of 3000
film 30 of 3000
film 31 of 3000
film 32 of 3000
film 33 of 3000
film 34 of 3000
film 35 of 3000
film 36 of 3000
film 37 of 3000
film 38 of 3000
film 39 of 3000
film 40 of 3000
film 41 of 3000
film 42 of 3000
film 43 of 3000
film 44 of 3000
film 45 of 3000
film 46 of 3000
film 47 of 3000
film 48 of 3000
film 49 of 3000
film 50 of 3000
film 51 of 3000
film 52 of 3000
film 53 of 3000
film 54 of 3000
film 55 of 3000
film 56 of 3000
film 57 of 3000
film 58 of 3000
film 59 of 3000
film 60 of 3000
film 61 of 3000
film 62 of 3000
film 63 of 3000
f

film 490 of 3000
film 491 of 3000
film 492 of 3000
film 493 of 3000
film 494 of 3000
film 495 of 3000
film 496 of 3000
film 497 of 3000
film 498 of 3000
film 499 of 3000
film 500 of 3000
film 501 of 3000
film 502 of 3000
film 503 of 3000
film 504 of 3000
film 505 of 3000
film 506 of 3000
film 507 of 3000
film 508 of 3000
film 509 of 3000
film 510 of 3000
film 511 of 3000
film 512 of 3000
film 513 of 3000
film 514 of 3000
film 515 of 3000
film 516 of 3000
film 517 of 3000
film 518 of 3000
film 519 of 3000
film 520 of 3000
film 521 of 3000
film 522 of 3000
film 523 of 3000
film 524 of 3000
film 525 of 3000
film 526 of 3000
film 527 of 3000
film 528 of 3000
film 529 of 3000
film 530 of 3000
film 531 of 3000
film 532 of 3000
film 533 of 3000
film 534 of 3000
film 535 of 3000
film 536 of 3000
film 537 of 3000
film 538 of 3000
film 539 of 3000
film 540 of 3000
film 541 of 3000
film 542 of 3000
film 543 of 3000
film 544 of 3000
film 545 of 3000
film 546 of 3000
film 547 of 3000
film 548 of 30

film 972 of 3000
film 973 of 3000
film 974 of 3000
film 975 of 3000
film 976 of 3000
film 977 of 3000
film 978 of 3000
film 979 of 3000
film 980 of 3000
film 981 of 3000
film 982 of 3000
film 983 of 3000
film 984 of 3000
film 985 of 3000
film 986 of 3000
film 987 of 3000
film 988 of 3000
film 989 of 3000
film 990 of 3000
film 991 of 3000
film 992 of 3000
film 993 of 3000
film 994 of 3000
film 995 of 3000
film 996 of 3000
film 997 of 3000
film 998 of 3000
film 999 of 3000
film 1000 of 3000
film 1001 of 3000
film 1002 of 3000
film 1003 of 3000
film 1004 of 3000
film 1005 of 3000
film 1006 of 3000
film 1007 of 3000
film 1008 of 3000
film 1009 of 3000
film 1010 of 3000
film 1011 of 3000
film 1012 of 3000
film 1013 of 3000
film 1014 of 3000
film 1015 of 3000
film 1016 of 3000
film 1017 of 3000
film 1018 of 3000
film 1019 of 3000
film 1020 of 3000
film 1021 of 3000
film 1022 of 3000
film 1023 of 3000
film 1024 of 3000
film 1025 of 3000
film 1026 of 3000
film 1027 of 3000
film 1028 of 3000
fi

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: received Inspector.detached event
  (Session info: chrome=105.0.5195.102)
Stacktrace:
Backtrace:
	Ordinal0 [0x008DDF13+2219795]
	Ordinal0 [0x00872841+1779777]
	Ordinal0 [0x0078423D+803389]
	Ordinal0 [0x00777DF8+753144]
	Ordinal0 [0x00777AF6+752374]
	Ordinal0 [0x0077706A+749674]
	Ordinal0 [0x00775F94+745364]
	Ordinal0 [0x00776528+746792]
	Ordinal0 [0x007806F5+788213]
	Ordinal0 [0x007778FD+751869]
	Ordinal0 [0x0077853F+755007]
	Ordinal0 [0x00777B11+752401]
	Ordinal0 [0x0077706A+749674]
	Ordinal0 [0x00775F94+745364]
	Ordinal0 [0x007763FD+746493]
	Ordinal0 [0x00785A8E+809614]
	Ordinal0 [0x007DF87D+1177725]
	Ordinal0 [0x007CE7FC+1107964]
	Ordinal0 [0x007DF192+1175954]
	Ordinal0 [0x007CE616+1107478]
	Ordinal0 [0x007A7F89+950153]
	Ordinal0 [0x007A8F56+954198]
	GetHandleVerifier [0x00BD2CB2+3040210]
	GetHandleVerifier [0x00BC2BB4+2974420]
	GetHandleVerifier [0x00976A0A+565546]
	GetHandleVerifier [0x0

In [10]:
# display a random entry from the dictionary to check it's working
keith = random.choice(list(film_dict.keys()))
print(keith)
print(film_dict[keith])

dune(1984)
{'id': 'tt0087182', 'punc_name': 'Dune (1984)', 'tags': ['space travel', 'desert planet', 'water', 'dune', 'giant worm', 'spice', 'father proud of son', 'facing fear', 'gagged woman', 'planetary romance', 'dream sequence', 'written by director', 'inner monologue', 'narrated by character', 'only son', 'waking dream', 'traitor', 'assassination attempt', 'assassination', 'murder', 'fake tooth', 'stabbed in the back', 'abduction', 'death of husband', 'emergency landing', 'dream', 'dreaming', 'ring', 'crying', 'birth', 'giving birth', 'death', 'duel', 'rain', 'weapons fire', 'loss of father', 'handheld weapon', 'drone', 'folding of space time', 'nobility', 'body suit', 'signet ring', 'concubine', 'sabotage', 'prisoner', 'gagged', 'psychotronic film', 'year 10191', '10190s', 'future', 'distant future', 'prophecy', 'empire', 'life extension', 'freedom', 'arrakis', 'guild', 'house atreides', 'house harkonnen', 'emperor of the universe', 'princess', 'sisterhood', 'eugenics', 'mentat'

In [11]:
# export the dictionary to an external json file for portability
with open(f'data/imdb_tag_game_{size}.json', 'w') as fp:
    json.dump(film_dict, fp)

In [12]:
film_names

['The Shawshank Redemption',
 'The Dark Knight',
 'Inception',
 'Fight Club',
 'Forrest Gump',
 'Pulp Fiction',
 'The Matrix',
 'The Lord of the Rings: The Fellowship of the Ring',
 'The Godfather',
 'The Lord of the Rings: The Return of the King',
 'Interstellar',
 'The Dark Knight Rises',
 'The Lord of the Rings: The Two Towers',
 'Seven',
 'Django Unchained',
 'Gladiator',
 'Batman Begins',
 'Inglourious Basterds',
 'The Silence of the Lambs',
 'The Wolf of Wall Street',
 'Avengers Assemble',
 'Saving Private Ryan',
 'Star Wars',
 "Schindler's List",
 'The Prestige',
 'The Departed',
 'Shutter Island',
 'The Green Mile',
 'Star Wars: Episode V - The Empire Strikes Back',
 'The Godfather: Part II',
 'Joker',
 'Memento',
 'Avatar',
 'Back to the Future',
 'Guardians of the Galaxy',
 'Titanic',
 'Leon',
 'American Beauty',
 'Goodfellas',
 'Pirates of the Caribbean: The Curse of the Black Pearl',
 'V for Vendetta',
 'American History X',
 'WALL·E',
 'Kill Bill: Vol. 1',
 'Avengers: Endg

In [13]:
film_dict.keys()

dict_keys(['shawshankredemption(1994)', 'darkknight(2008)', 'inception(2010)', 'fightclub(1999)', 'forrestgump(1994)', 'pulpfiction(1994)', 'matrix(1999)', 'lordofringsfellowshipofring(2001)', 'godfather(1972)', 'lordofringsreturnofking(2003)', 'interstellar(2014)', 'darkknightrises(2012)', 'lordofringstwotowers(2002)', 'seven(1995)', 'djangounchained(2012)', 'gladiator(2000)', 'batmanbegins(2005)', 'inglouriousbasterds(2009)', 'silenceoflambs(1991)', 'wolfofwallstreet(2013)', 'avengersassemble(2012)', 'savingprivateryan(1998)', 'starwars(1977)', 'schindlerslist(1993)', 'prestige(2006)', 'departed(2006)', 'shutterisland(2010)', 'greenmile(1999)', 'starwarsepisodevempirestrikesback(1980)', 'godfatherpartii(1974)', 'joker(I) (2019)', 'memento(2000)', 'avatar(2009)', 'backtofuture(1985)', 'guardiansofgalaxy(2014)', 'titanic(1997)', 'leon(1994)', 'americanbeauty(1999)', 'goodfellas(1990)', 'piratesofcaribbeancurseofblackpearl(2003)', 'vforvendetta(2005)', 'americanhistoryx(1998)', 'walle(2

In [14]:
browser.quit()